Try timestamp input with time HH:MM:SS

! timedelta(days=1) = timedelta(hours=24) -> doesn't change at midnight/partial days

Would do with more time:

Would find way to calculate calendar days vs. 24hr gaps to filter by TIME for any accounts repeating in a day -- OR -- would select later time (if timestamp occurs >1, select latest)

Find nearest date if date doesn't occur

In [13]:
import pandas as pd
from datetime import timedelta

In [14]:
# Read in data
# products = pd.read_csv("../Resources/products.csv")
# accounts = pd.read_csv("../Resources/accounts.csv")
ast_df = pd.read_csv("../Resources/account_state_transitions.csv")
# gpa = pd.read_csv("../Resources/group_product_associations.csv")
# groups = pd.read_csv("../Resources/groups.csv")
# payments = pd.read_csv("../Resources/payments.csv")

In [15]:
ast_df.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
0,3,2020-03-12 20:03:35.724574,2019-11-06 05:49:39.571392,2,CREATED,ENABLED,NaN
1,4,2020-03-12 20:03:35.724574,2019-11-25 22:20:59.150339,2,ENABLED,DISABLED,3.0
2,5,2020-03-12 20:03:36.042456,2019-11-26 10:22:36.571392,2,DISABLED,ENABLED,4.0
3,6,2020-03-12 20:03:36.042456,2019-11-29 11:03:32.150339,2,ENABLED,DISABLED,5.0
4,7,2020-03-12 20:03:36.098448,2019-12-03 10:08:03.571392,2,DISABLED,ENABLED,6.0


## 2) Devise a method of generating a table of all Accounts with their State as of any arbitrary timestamp in the past and the total time they had been in that state as of the timestamp.

### Limit Dataframe by Indicated Timestamp

In [16]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2020-03-01


In [17]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

In [18]:
# Change data type from string to datetime object
ast_df['started_when'] = pd.to_datetime(ast_df['started_when'])

In [19]:
# Create limited dataframe based on indicated timestamp 
df_limited = ast_df.loc[(ast_df['started_when'] >= timestamp_dt) & (ast_df['started_when'] < timestamp_dt + timedelta(days=1))]
df_limited.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
894,4689,2020-03-12 20:05:14.950510,2020-03-01 06:59:59.571392,34,DISABLED,UNLOCKED,4688.0
1009,6862,2020-03-12 20:05:33.177894,2020-03-01 08:29:39.571392,38,DISABLED,ENABLED,6861.0
1045,7272,2020-03-12 20:05:37.621790,2020-03-01 22:46:39.571392,39,DISABLED,ENABLED,7271.0
1112,7963,2020-03-12 20:05:46.723718,2020-03-01 20:41:24.340623,42,ENABLED,DISABLED,7962.0
1407,9576,2020-03-12 20:06:43.954110,2020-03-01 16:07:29.571392,60,DISABLED,ENABLED,9575.0


### Current State of Each Account by Indicated Timestamp

In [20]:
# Create dataframe to show current state of each account based on indicated timestamp
df_limited2 = df_limited[['started_when','account_id','to_state']]

# Sort by account_id
df_limited2 = df_limited2.sort_values('account_id')

# Reset index
df_limited2 = df_limited2.reset_index(drop=True)
df_limited2.head()

,started_when,account_id,to_state
0,2020-03-01 06:59:59.571392,34,UNLOCKED
1,2020-03-01 08:29:39.571392,38,ENABLED
2,2020-03-01 22:46:39.571392,39,ENABLED
3,2020-03-01 20:41:24.340623,42,DISABLED
4,2020-03-01 22:57:43.571392,55,ENABLED


In [21]:
# Show in dataframe, any accounts occuring more than once during indicated timestamp

account_ids = []
df_repeat = pd.DataFrame(columns=['started_when', 'account_id', 'to_state'])

for i in range(0,len(df_limited2['account_id'])):
    account_id = df_limited2['account_id'][i]
    if account_id in account_ids:
        df_temp = df_limited2.loc[df_limited2['account_id'] == account_id]
        df_repeat = df_repeat.append(df_temp)
    else:
        account_ids.append(account_id)

df_repeat

,started_when,account_id,to_state
21,2020-03-01 19:49:56.971392,168,DISABLED
22,2020-03-01 10:55:42.571392,168,ENABLED


### Total Time in State as of Indicated Timestamp